In [10]:
import yfinance as yf
import pandas as pd
import polars as pl
import yahoo_fin.stock_info as si
import datetime as dt
from functools import reduce
from datetime import datetime, timedelta
from yahoo_fin import *
from pandas_datareader import data as pdr

W – means there are outstanding warrants.
R – means there is some kind of “rights” issue.
P – means “First Preferred Issue”. Preferred stocks are separate entity.
Q - means bankruptcy.

In [11]:
# deleting stock symbols/tickers within the nasdaq listing
nasdaq = pd.DataFrame(si.tickers_nasdaq())
sym = set( symbol for symbol in nasdaq[0].values.tolist() )

# deleting the inactive tickers by suffix
my_list = ['C', 'W', 'T', 'R', 'P', 'Q', 'Z', 'I', '.A']

# variable for tickers to be saved and disgaurded
del_set = set()
sav_set = set()

# loop to id the suffix and disgaurd
for symbol in sym:
    if len(symbol) > 4 and symbol[-1] in my_list:
        del_set.add(symbol)
    else:
        sav_set.add(symbol)

tickers = sorted(sav_set) # [:10]

In [13]:
# Initialize an empty list to store stock movement data
movementlist = []

# Loop through each ticker symbol
for stock in tickers:
    # Download the stock's 5-day price history using yfinance
    thestock = yf.download(tickers=stock, period="5d", interval="1d", ignore_tz=True, prepost=False, progress=False)

    # Find the minimum low price and the maximum high price over the 5-day period
    low = thestock['Low'].min()
    high = thestock['High'].max()

    # Calculate the percentage change between the high and low prices
    if low != 0:
        deltapercent = 100 * (high - low) / low
    else:
        deltapercent = 0

    # Fetch the opening price of the stock
    # If there are at least 5 days of history, also fetch the closing price of the stock
    # Otherwise, use the opening price as the closing price
    if len(thestock) >= 5:
        Close = thestock.iloc[4]["Close"]
        Open = thestock.iloc[0]["Open"]
    else:
        Close = Open = thestock.iloc[0]["Open"]

    # Calculate the percentage change in price from opening to closing
    # If the opening price is zero (to avoid division by zero), set the price change to zero
    deltaprice = 100 * (Close - Open) / Open if Open != 0 else 0

    # Append the stock symbol, high price, low price, percentage change, and price change to the movement list
    pair = [stock, high, low, deltapercent, deltaprice]
    movementlist.append(pair)


1 Failed download:
['THAR']: Exception('%ticker%: No price data found, symbol may be delisted (period=5d)')


IndexError: single positional indexer is out-of-bounds

In [ ]:
for entry in movementlist:
    if entry[1]>float(100):
        print(entry)

In [ ]:
# High risers:
def lookup_stockinfo(thestock):
    try:
        return thestock.info
    except IndexError:
        return 0
    
cutoff=float(80)
        
for entry in movementlist:
    if entry[2]>cutoff:
        print("\n"+ str(entry))
        thestock = yf.download(str(entry[0]))
    if entry[0]=='HJLIW':
            print("no info")
    else:
        a = lookup_stockinfo(thestock)
    if a == 0:
        print("no info")
    else:
        if a is None:
            print("no info")
        else:
            if a == "":
                print("no")
            else:
                print(a)
                print('Up '+ str(entry[2]) + "%")
                print(str(a['sector']))
                print(str(a['longBusinessSummary']))
                print("year high "+ str(a['fiftyTwoWeekHigh']))

<bound method DataFrame.info of                  Open       High        Low      Close  Adj Close  Volume
Date                                                                     
2023-05-22  24.639999  24.660000  24.530001  24.597000  24.597000   10100
2023-05-23  24.230000  24.340000  23.930000  23.990000  23.990000   12600
2023-05-24  24.021000  24.049999  23.809999  24.049999  24.049999    6300
2023-05-25  24.075001  24.400000  24.075001  24.273001  24.273001    7900
2023-05-26  24.530001  24.850000  24.530001  24.850000  24.850000    6200>
Up 1.4800000190734863%


TypeError: 'method' object is not subscriptable

In [ ]:
# Convert the movement list into a DataFrame
movement = pd.DataFrame(movementlist, columns=['stock', 'high', 'low', 'deltapercent', 'deltaprice'])
movement

ValueError: 6 columns passed, passed data had 5 columns

In [ ]:
# creating a csv file of the DataFrame
# movement.to_csv('latest_movement.csv', header=True, index=True, encoding='utf-8')

: 